<a href="https://colab.research.google.com/github/max1nehour/Carotid-Artery-Image-Classification-Model/blob/main/Catboost_Combined_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install seaborn

In [ ]:
pip install catboost

In [ ]:
pip install scikit-learn==1.0.2

In [ ]:
pip install --upgrade scikit-learn

In [ ]:
pip install imblearn

In [ ]:
import psutil
import os

# Get process memory info
process = psutil.Process(os.getpid())
memory_info = process.memory_info()

# Convert from bytes to MB
memory_used_mb = memory_info.rss / 1024 ** 2

print(f"Memory used: {memory_used_mb:.2f} MB")


Memory used: 58.96 MB


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split,  StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree
from numpy import sort
from statistics import mean
from scipy.stats import chi2_contingency
from catboost import Pool, CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.cluster import KMeans
from collections import Counter

In [ ]:

def find_plaque(severity):
    if int(severity)>1:
        return 1
    else:
        return 0

def print_feature(model, df):
    features = model.feature_importances_

    # 獲取列名並轉換為列表
    gene_list = df

    # 過濾掉重要性非常低的特徵
    threshold = 0  # 設定過濾不重要特徵的門檻
    indices = np.where(features > threshold)[0]
    filtered_importances = features[indices]
    filtered_feature_names = np.array(gene_list)[indices]

    # 按重要性排序
    sorted_idx = np.argsort(filtered_importances)[::-1]
    filtered_importances = filtered_importances[sorted_idx]
    filtered_feature_names = filtered_feature_names[sorted_idx]

     # 创建 DataFrame
    fs_df = pd.DataFrame({
        'Feature Name': filtered_feature_names,
        'Importance': filtered_importances
    })

    # 繪製帶有 hue 的條形圖
    plt.figure(figsize=(6, 10))
    sns.barplot(x=filtered_importances, y=filtered_feature_names, hue=filtered_importances, palette='crest', dodge=False)

    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.title('Feature Importances')
    plt.yticks(fontsize=11)
    plt.legend(title='Importance Level')
    plt.show()

    return fs_df


In [ ]:
df1 = pd.read_csv('base_allplq.csv')
df2 = pd.read_csv('follow_allplq.csv')
df = pd.concat([df1, df2])
df = df.drop(columns=['SIT_3_SYSTOLIC_PRESSURE', 'SIT_3_DIASTOLIC_PRESSURE','SMK_2ND_PLACE4_HR','RCCA_Plaque','RICA_Plaque','LCCA_Plaque','LICA_Plaque'
                     ,'SURVEY_DATE','Release_No','頸超總結'])
pd.set_option('display.max_rows', None)
print(df.isnull().sum())

df['BODY_FAT_RATE'] = df['BODY_FAT_RATE'].fillna(df['BODY_FAT_RATE'].median())
df = df.dropna()
df['Plaque'] = df['Severity'].apply(lambda x: find_plaque(x))

print(df.shape)
print(df.columns.to_list())
print(df.isnull().sum())


/tmp/ipykernel_1490/1901660766.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('base_allplq.csv')
/tmp/ipykernel_1490/1901660766.py:2: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('follow_allplq.csv')


PA_UIKey                          0
RCCA_MAX_FLOW                     8
RCCA_AVG_FLOW                    20
RCCA_MIN_FLOW                     6
RCCA_MAX_AVG_FLOW                20
RCCA_PULSE                       20
RCCA_RESISTANCE                   6
RCCA_DIAMETER                    10
RCCA_STATUS                       6
RCCA_IMT                         22
RICA_MAX_FLOW                    12
RICA_AVG_FLOW                    22
RICA_MIN_FLOW                     8
RICA_MAX_AVG_FLOW                22
RICA_PULSE                       22
RICA_RESISTANCE                  10
RICA_DIAMETER                    32
右ICA正常與否                          6
LCCA_MAX_FLOW                    10
LCCA_AVG_FLOW                    20
LCCA_MIN_FLOW                     8
LCCA_MAX_AVG_FLOW                20
LCCA_PULSE                       20
LCCA_RESISTANCE                  10
LCCA_DIAMETER                    16
左CCA正常與否                          6
LCCA_IMT                         30
LICA_MAX_FLOW               

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, SMOTENC, SVMSMOTE
import numpy as np
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.preprocessing import StandardScaler

##################### ALL ARTERY TO ALL DIAGNOSIS #################

#Define Input/ Output (array type)
feature = base.drop(labels=['PA_UIKey','Plaque','Severity','FOLLOW','RCCA_STATUS','右ICA正常與否','左CCA正常與否','左ICA正常與否'],axis=1)
feature = feature.columns.to_list()

#df = df.drop(columns=['SIT_3_SYSTOLIC_PRESSURE', 'SIT_3_DIASTOLIC_PRESSURE','SMK_2ND_PLACE4_HR','RCCA_Plaque','RICA_Plaque','LCCA_Plaque','LICA_Plaque'
                     #,'PA_UIKey','SURVEY_DATE','Release_No','頸超總結'])
#####################################################################
X_train = base.drop(columns=['PA_UIKey','Plaque','Severity','FOLLOW','RCCA_STATUS','右ICA正常與否','左CCA正常與否','左ICA正常與否']).values
y_train = base['Plaque'].values
X_test = follow.drop(columns=['PA_UIKey','Plaque','Severity','FOLLOW','RCCA_STATUS','右ICA正常與否','左CCA正常與否','左ICA正常與否']).values
y_test = follow['Plaque'].values


# Combine X_train and X_test, and y_train and y_test
X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

# Count the occurrences of 1s and 0s in y_train
unique_train, counts_train = np.unique(y_train, return_counts=True)
# Print the counts
print("y_train = 1:", counts_train[unique_train == 1][0])
print("y_train = 0:", counts_train[unique_train == 0][0])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Oversample the PCA-transformed training data
from imblearn.over_sampling import SMOTE,SMOTEN

smote_nc = SVMSMOTE(random_state=0)
method = smote_nc
X_resampled, y_resampled = method.fit_resample(X_train, y_train)

# Count the occurrences of 1s and 0s in y_train
unique_train, counts_train = np.unique(y_resampled, return_counts=True)
# Print the counts
print("y_train = 1:", counts_train[unique_train == 1][0])
print("y_train = 0:", counts_train[unique_train == 0][0])

#####################################################################
# Check the dimensions of X
print("Dimensions of X train:", X_resampled.shape)
print("Dimensions of X test:", X_test.shape)


# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.01,
                           depth=6,l2_leaf_reg=3.0,
                           loss_function='MultiClass',rsm=0.8,verbose=100,
                          early_stopping_rounds=50 )
model.fit(X_resampled, y_resampled)
print_feature(model, feature)

#prediction
probability_predictions = model.predict_proba(X_test)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

#評估報表
print('--------------------REPORT: {}---------------------\n')
print(classification_report(y_test, y_pred))

#################################################################################################################################

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Predict probabilities for the ROC curve
y_prob = model.predict_proba(X_test)[:, 1]  # Probability estimates for the positive class

# Compute the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, color='seagreen', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='midnightblue', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right',fontsize =16)
plt.show()

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)


# Plot confusion matrix
plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()



NameError: name 'base' is not defined

In [ ]:
###################### 10 fold ######################

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, f1_score
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imblearn_Pipeline

#Define Input/ Output (array type)
feature = base.drop(labels=['PA_UIKey','Plaque','Severity','FOLLOW','RCCA_STATUS','右ICA正常與否','左CCA正常與否','左ICA正常與否'],axis=1)
feature = feature.columns.to_list()
# Your data preparation
X_train = base.drop(columns=['PA_UIKey','Plaque','Severity','FOLLOW','RCCA_STATUS','右ICA正常與否','左CCA正常與否','左ICA正常與否']).values
y_train = base['Plaque'].values
X_test = follow.drop(columns=['PA_UIKey','Plaque','Severity','FOLLOW','RCCA_STATUS','右ICA正常與否','左CCA正常與否','左ICA正常與否']).values
y_test = follow['Plaque'].values

# Combine X_train and X_test, and y_train and y_test
X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

# Count the occurrences of 1s and 0s in y_train
unique_train, counts_train = np.unique(y_train, return_counts=True)
print("y_train = 1:", counts_train[unique_train == 1][0])
print("y_train = 0:", counts_train[unique_train == 0][0])

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define model
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.01,
                           depth=6,
                           l2_leaf_reg=3.0,
                           loss_function='Logloss',
                           rsm=0.8,
                           verbose=100,
                           early_stopping_rounds=50)

# Define evaluation metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'auc': make_scorer(roc_auc_score, needs_proba=True),
    'f1': make_scorer(f1_score)
}

# Set up 10-fold cross-validation
cv = KFold(n_splits=10, shuffle=True, random_state=0)

# Use a pipeline for scaling and oversampling
pipeline = imblearn_Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=0)),
    ('classifier', model)
])

# Perform cross-validation
cv_results = cross_validate(pipeline, X, y, cv=cv, scoring=scoring, return_train_score=False)

# Print results
print(f"{cv_results['test_accuracy']:.4f}")
print(f"{cv_results['test_auc']:.4f}")

print(f'Cross-Validation Accuracy: {np.mean(cv_results["test_accuracy"]):.4f} ± {np.std(cv_results["test_accuracy"]):.4f}')
print(f'Cross-Validation AUC: {np.mean(cv_results["test_auc"]):.4f} ± {np.std(cv_results["test_auc"]):.4f}')
print(f'Cross-Validation F1 Score: {np.mean(cv_results["test_f1"]):.4f} ± {np.std(cv_results["test_f1"]):.4f}')

# Train final model on entire dataset
pipeline.fit(X, y)
print_feature(model, feature)
# Predictions on test data
y_pred = pipeline.predict(X_test)
probability_predictions = pipeline.predict_proba(X_test)

# Compute metrics for test data
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, probability_predictions[:, 1])
f1 = f1_score(y_test, y_pred)

print('--------------------REPORT---------------------')
print(f'Accuracy: {accuracy:.4f}')
print(f'AUC: {auc:.4f}')
print(f'F1 Score: {f1:.4f}')
print(classification_report(y_test, y_pred))


In [ ]:
##################### ALL ARTERY TO Apoplexia #################

#Define Input/ Output (array type)
feature = base.drop(labels=['Severity','FOLLOW','APOPLEXIA_SELF','RCCA_STATUS','右ICA正常與否','左CCA正常與否','左ICA正常與否'],axis=1)
feature = feature.columns.to_list()

#df = df.drop(columns=['SIT_3_SYSTOLIC_PRESSURE', 'SIT_3_DIASTOLIC_PRESSURE','SMK_2ND_PLACE4_HR','RCCA_Plaque','RICA_Plaque','LCCA_Plaque','LICA_Plaque'
                     #,'PA_UIKey','SURVEY_DATE','Release_No','頸超總結'])
#####################################################################
X_train = base.drop(columns=['Severity','FOLLOW','APOPLEXIA_SELF','RCCA_STATUS','右ICA正常與否','左CCA正常與否','左ICA正常與否']).values
y_train = base['APOPLEXIA_SELF'].values
X_test = follow.drop(columns=['Severity','FOLLOW','APOPLEXIA_SELF','RCCA_STATUS','右ICA正常與否','左CCA正常與否','左ICA正常與否']).values
y_test = follow['APOPLEXIA_SELF'].values


# Combine X_train and X_test, and y_train and y_test
X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

# Count the occurrences of 1s and 0s in y_train
# Count the number of 1s in y_train
count_ones = sum(y_train)
count_zeros = len(y_train) - count_ones
# Print the counts
print("y_train = 1:", count_ones)
print("y_train = 0:", count_zeros)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Oversample the PCA-transformed training data
from imblearn.over_sampling import SMOTE,SMOTEN

# smote_nc = SVMSMOTE(random_state=0)
# method = smote_nc
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)


# Print the counts
print("y_train = 1:", counts_train[unique_train == 1][0])
print("y_train = 0:", counts_train[unique_train == 0][0])

#####################################################################
# Check the dimensions of X
print("Dimensions of X train:", X_resampled.shape)
print("Dimensions of X test:", X_test.shape)


# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.01,
                           depth=6,l2_leaf_reg=3.0,
                           loss_function='MultiClass',rsm=0.8,verbose=100,
                          early_stopping_rounds=50 )
model.fit(X_resampled,  y_resampled )
fs_df = print_feature(model, feature)

from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_resampled,  y_resampled, cv=5, scoring='f1')
print(f"Cross-validated F1 scores: {scores}")

#prediction
probability_predictions = model.predict_proba(X_test)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

#評估報表
print('--------------------REPORT: {}---------------------\n')
print(classification_report(y_test, y_pred))

#################################################################################################################################

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Predict probabilities for the ROC curve
y_prob = model.predict_proba(X_test)[:, 1]  # Probability estimates for the positive class

# Compute the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, color='seagreen', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='midnightblue', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right',fontsize =16)
plt.show()

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)


# Plot confusion matrix
plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_ )[::-1]

fs = []
for thresh in thresholds:
    if thresh <=0:
        break
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_resampled)

    # train model
    selection_model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.01,
                           depth=6,l2_leaf_reg=3.0,
                           loss_function='MultiClass',rsm=0.8,verbose=100,
                          early_stopping_rounds=50 )
    selection_model.fit(select_X_train, y_train)

    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    accuracy = accuracy_score(y_test, y_pred)
#     fs.append((thresh, select_X_train.shape[1], accuracy*100.0))
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))
#     fs_df = pd.DataFrame(fs, columns=['Thresh','Feature Counts', 'Accuracy'])
    fs_df.to_csv('Apoplexia_feature_selection.csv')


In [ ]:
fcounts = fs_df['Feature Counts'].to_list()
acc = fs_df['Accuracy'].to_list()

# Convert 1D data to a 2D format
data_2d = np.array(acc).reshape(1, -1)  # Single-row matrix
accuracy_data_normalized = data_2d / 100
accuracy_data_rounded = np.round(accuracy_data_normalized, 2)
data_2d = accuracy_data_rounded.T
# Define custom labels for x and y axes
xticks_labels = ['Accuracy']
yticks_labels = fcounts

plt.figure(figsize=(1, 20))
# Create a heatmap
plt.xticks(fontsize=14)
sns.heatmap(data_2d, cmap='crest', annot=True, cbar=True, xticklabels=xticks_labels, yticklabels=yticks_labels)
plt.title('Accuracy of each artery')
plt.show()

In [ ]:
################# RCCA ################################
#Define Input/ Output (array type)
rcca = df.iloc[:, list(range(0, 10)) + list(range(35, 59))]
rcca['RCCA_STATUS'] = rcca['RCCA_STATUS'].replace('異常', 1)
rcca['RCCA_STATUS'] = rcca['RCCA_STATUS'].replace('正常', 0)
rcca['RCCA_STATUS'] = rcca['RCCA_STATUS'].replace('1.5', 1)
print(df.columns)
print(rcca.columns)

base = rcca[rcca['FOLLOW']=='Baseline']
follow = rcca[rcca['FOLLOW']=='Follow 1']

feature = base.drop(labels=['PA_UIKey','Severity','FOLLOW','RCCA_STATUS'],axis=1)
feature = feature.columns.to_list()
print(feature)

#df = df.drop(columns=['SIT_3_SYSTOLIC_PRESSURE', 'SIT_3_DIASTOLIC_PRESSURE','SMK_2ND_PLACE4_HR','RCCA_Plaque','RICA_Plaque','LCCA_Plaque','LICA_Plaque'
                     #,'PA_UIKey','SURVEY_DATE','Release_No','頸超總結'])
#####################################################################
X_train = base.drop(columns=['PA_UIKey','Severity','FOLLOW','RCCA_STATUS']).values
y_train = base['RCCA_STATUS'].values
X_test = follow.drop(columns=['PA_UIKey','Severity','FOLLOW','RCCA_STATUS']).values
y_test = follow['RCCA_STATUS'].values
print(y_train)

# Combine X_train and X_test, and y_train and y_test
X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

# Count the occurrences of 1s and 0s in y_train
count_ones = sum(y_train)
count_zeros = len(y_train) - count_ones
# Print the counts
print("y_train = 1:", count_ones)
print("y_train = 0:", count_zeros)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Oversample the PCA-transformed training data
from imblearn.over_sampling import SMOTE,SMOTEN

# smote_nc = SVMSMOTE(random_state=0)
# method = smote_nc
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)


# Print the counts
print("y_train = 1:", counts_train[unique_train == 1][0])
print("y_train = 0:", counts_train[unique_train == 0][0])

#####################################################################
# Check the dimensions of X
print("Dimensions of X train:", X_resampled.shape)
print("Dimensions of X test:", X_test.shape)


# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.01,
                           depth=6,l2_leaf_reg=3.0,
                           loss_function='MultiClass',rsm=0.8,verbose=100,
                          early_stopping_rounds=50 )
model.fit(X_resampled, y_resampled)

probability_predictions = model.predict_proba(X_test)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

fs_df = print_feature(model, feature)

#評估報表
print('--------------------REPORT: {}---------------------\n')
print(classification_report(y_test, y_pred))


#################################################################################################################################

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Predict probabilities for the ROC curve
y_prob = model.predict_proba(X_test)[:, 1]  # Probability estimates for the positive class

# Compute the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, color='seagreen', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='midnightblue', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right',fontsize =16)
plt.show()

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)


# Plot confusion matrix
plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_ )[::-1]

fs = []
for thresh in thresholds:
    if thresh <=0:
        break
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)

    # train model
    selection_model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.01,
                           depth=6,l2_leaf_reg=3.0,
                           loss_function='MultiClass',rsm=0.8,verbose=100,
                          early_stopping_rounds=50 )
    selection_model.fit(select_X_train, y_train)

    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    accuracy = accuracy_score(y_test, y_pred)
    fs.append((thresh, select_X_train.shape[1], accuracy*100.0))
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))
    fs_df.to_csv('RCCA_feature_selection.csv')

In [ ]:
################# RICA ################################
#Define Input/ Output (array type)

rcca = df.iloc[:, list(range(10, 18)) + list(range(35, 59))]
rcca['右ICA正常與否'] = rcca['右ICA正常與否'].replace('異常', 1)
rcca['右ICA正常與否'] = rcca['右ICA正常與否'].replace('正常', 0)
rcca['右ICA正常與否'] = rcca['右ICA正常與否'].replace('1.5', 1)
print(df.columns)
print(rcca.columns)

base = rcca[rcca['FOLLOW']=='Baseline']
follow = rcca[rcca['FOLLOW']=='Follow 1']

feature = base.drop(labels=['Severity','FOLLOW','右ICA正常與否'],axis=1)
feature = feature.columns.to_list()
print(feature)

#df = df.drop(columns=['SIT_3_SYSTOLIC_PRESSURE', 'SIT_3_DIASTOLIC_PRESSURE','SMK_2ND_PLACE4_HR','RCCA_Plaque','RICA_Plaque','LCCA_Plaque','LICA_Plaque'
                     #,'PA_UIKey','SURVEY_DATE','Release_No','頸超總結'])
#####################################################################
X_train = base.drop(columns=['Severity','FOLLOW','右ICA正常與否']).values
y_train = base['右ICA正常與否'].values
X_test = follow.drop(columns=['Severity','FOLLOW','右ICA正常與否']).values
y_test = follow['右ICA正常與否'].values
print(y_train)

# Combine X_train and X_test, and y_train and y_test
X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

# Count the occurrences of 1s and 0s in y_train
count_ones = sum(y_train)
count_zeros = len(y_train) - count_ones
# Print the counts
print("y_train = 1:", count_ones)
print("y_train = 0:", count_zeros)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Oversample the PCA-transformed training data
from imblearn.over_sampling import SMOTE,SMOTEN

# smote_nc = SVMSMOTE(random_state=0)
# method = smote_nc
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)


# Print the counts
print("y_train = 1:", counts_train[unique_train == 1][0])
print("y_train = 0:", counts_train[unique_train == 0][0])

#####################################################################
# Check the dimensions of X
print("Dimensions of X train:", X_resampled.shape)
print("Dimensions of X test:", X_test.shape)


# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.01,
                           depth=6,l2_leaf_reg=3.0,
                           loss_function='MultiClass',rsm=0.8,verbose=100,
                          early_stopping_rounds=50 )
model.fit(X_resampled, y_resampled)

probability_predictions = model.predict_proba(X_test)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

fs_df = print_feature(model, feature)

#評估報表
print('--------------------REPORT: {}---------------------\n')
print(classification_report(y_test, y_pred))
#################################################################################################################################

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Predict probabilities for the ROC curve
y_prob = model.predict_proba(X_test)[:, 1]  # Probability estimates for the positive class

# Compute the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, color='seagreen', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='midnightblue', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right',fontsize =16)
plt.show()

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)


# Plot confusion matrix
plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_ )[::-1]

fs = []
for thresh in thresholds:
    if thresh <=0:
        break
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)

    # train model
    selection_model = CatBoostClassifier(iterations=100,
                           learning_rate=0.01,
                           depth=6,l2_leaf_reg=3.0,
                           loss_function='MultiClass',rsm=0.8,verbose=100,
                          early_stopping_rounds=50 )
    selection_model.fit(select_X_train, y_train)

    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    accuracy = accuracy_score(y_test, y_pred)
    fs.append((thresh, select_X_train.shape[1], accuracy*100.0))
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))
    fs_df.to_csv('RICA_feature_selection.csv')

In [ ]:
################# LCCA ################################
#Define Input/ Output (array type)
lcca = df.iloc[:, list(range(18, 27)) + list(range(35, 59))]
lcca = lcca.dropna()
lcca['左CCA正常與否'] = lcca['左CCA正常與否'].replace('異常', 1)
lcca['左CCA正常與否'] =lcca['左CCA正常與否'].replace('正常', 0)
lcca['左CCA正常與否'] = lcca['左CCA正常與否'].replace('異常無', 1)

print(df.columns)
print(lcca.columns)

base = lcca[lcca['FOLLOW']=='Baseline']
follow = lcca[lcca['FOLLOW']=='Follow 1']

feature = base.drop(labels=['Severity','FOLLOW','左CCA正常與否'],axis=1)
feature = feature.columns.to_list()
print(feature)

#df = df.drop(columns=['SIT_3_SYSTOLIC_PRESSURE', 'SIT_3_DIASTOLIC_PRESSURE','SMK_2ND_PLACE4_HR','RCCA_Plaque','RICA_Plaque','LCCA_Plaque','LICA_Plaque'
                     #,'PA_UIKey','SURVEY_DATE','Release_No','頸超總結'])
#####################################################################
X_train = base.drop(columns=['Severity','FOLLOW','左CCA正常與否']).values
y_train = base['左CCA正常與否'].values
X_test = follow.drop(columns=['Severity','FOLLOW','左CCA正常與否']).values
y_test = follow['左CCA正常與否'].values

# Combine X_train and X_test, and y_train and y_test
X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

# Check if there are values other than 0 and 1
unique_values = base['左CCA正常與否'].unique()
non_binary_values = [val for val in unique_values if val not in (0, 1)]
if non_binary_values:
    print(f"y_train contains values other than 0 and 1: {non_binary_values}")
else:
    print("y_train contains only 0 and 1.")

# Count the occurrences of 1s and 0s in y_train
count_ones = sum(y_train)
count_zeros = len(y_train) - count_ones
# Print the counts
print("y_train = 1:", count_ones)
print("y_train = 0:", count_zeros

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Oversample the PCA-transformed training data
from imblearn.over_sampling import SMOTE,SMOTEN

# smote_nc = SVMSMOTE(random_state=0)
# method = smote_nc
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

# Count the occurrences of 1s and 0s in y_train
count_ones = sum(y_resampled)
count_zeros = len(y_resampled) - count_ones
# Print the counts
print("y_train = 1:", count_ones)
print("y_train = 0:", count_zeros)


#####################################################################
# Check the dimensions of X
print("Dimensions of X train:", X_resampled.shape)
print("Dimensions of X test:", X_test.shape)


# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.01,
                           depth=6,l2_leaf_reg=3.0,
                           loss_function='MultiClass',rsm=0.8,verbose=100,
                          early_stopping_rounds=50 )
model.fit(X_resampled, y_resampled)

probability_predictions = model.predict_proba(X_test)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

fs_df = print_feature(model, feature)

#評估報表
print('--------------------REPORT: {}---------------------\n')
print(classification_report(y_test, y_pred))
#################################################################################################################################

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Predict probabilities for the ROC curve
y_prob = model.predict_proba(X_test)[:, 1]  # Probability estimates for the positive class

# Compute the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, color='seagreen', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='midnightblue', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right',fontsize =16)
plt.show()

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)


# Plot confusion matrix
plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_ )[::-1]

fs = []
for thresh in thresholds:
    if thresh <=0:
        break
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)

    # train model
    selection_model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.01,
                           depth=6,l2_leaf_reg=3.0,
                           loss_function='MultiClass',rsm=0.8,verbose=100,
                          early_stopping_rounds=50 )
    selection_model.fit(select_X_train, y_train)

    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    accuracy = accuracy_score(y_test, y_pred)
    fs.append((thresh, select_X_train.shape[1], accuracy*100.0))
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))
    fs_df.to_csv('LCCA_feature_selection.csv')

In [ ]:
################# LICA ################################

#Define Input/ Output (array type)
lica = df.iloc[:, list(range(27, 35)) + list(range(35, 59))]
lica = lica.dropna()
lica['左ICA正常與否'] = lica['左ICA正常與否'].replace('異常', 1)
lica['左ICA正常與否'] = lica['左ICA正常與否'].replace('正常', 0)
lica['左ICA正常與否'] = lica['左ICA正常與否'].replace('異常無', 1)

print(df.columns)
print(lica.columns)

base = lica[lica['FOLLOW']=='Baseline']
follow = lica[lica['FOLLOW']=='Follow 1']

feature = base.drop(labels=['Severity','FOLLOW','左ICA正常與否'],axis=1)
feature = feature.columns.to_list()
print(feature)

#df = df.drop(columns=['SIT_3_SYSTOLIC_PRESSURE', 'SIT_3_DIASTOLIC_PRESSURE','SMK_2ND_PLACE4_HR','RCCA_Plaque','RICA_Plaque','LCCA_Plaque','LICA_Plaque'
                     #,'PA_UIKey','SURVEY_DATE','Release_No','頸超總結'])
#####################################################################
X_train = base.drop(columns=['Severity','FOLLOW','左ICA正常與否']).values
y_train = base['左ICA正常與否'].values
X_test = follow.drop(columns=['Severity','FOLLOW','左ICA正常與否']).values
y_test = follow['左ICA正常與否'].values

# Combine X_train and X_test, and y_train and y_test
X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

# Check if there are values other than 0 and 1
unique_values = base['左ICA正常與否'].unique()
non_binary_values = [val for val in unique_values if val not in (0, 1)]
if non_binary_values:
    print(f"y_train contains values other than 0 and 1: {non_binary_values}")
else:
    print("y_train contains only 0 and 1.")

# Count the occurrences of 1s and 0s in y_train
count_ones = sum(y_train)
count_zeros = len(y_train) - count_ones
# Print the counts
print("y_train = 1:", count_ones)
print("y_train = 0:", count_zeros)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Oversample the PCA-transformed training data
from imblearn.over_sampling import SMOTE,SMOTEN

# smote_nc = SVMSMOTE(random_state=0)
# method = smote_nc
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

# Count the occurrences of 1s and 0s in y_train
count_ones = sum(y_resampled)
count_zeros = len(y_resampled) - count_ones
# Print the counts
print("y_train = 1:", count_ones)
print("y_train = 0:", count_zeros)


#####################################################################
# Check the dimensions of X
print("Dimensions of X train:", X_resampled.shape)
print("Dimensions of X test:", X_test.shape)


# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.01,
                           depth=6,l2_leaf_reg=3.0,
                           loss_function='MultiClass',rsm=0.8,verbose=100,
                          early_stopping_rounds=50 )
model.fit(X_resampled, y_resampled)

probability_predictions = model.predict_proba(X_test)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

fs_df = print_feature(model, feature)

#評估報表
print('--------------------REPORT: {}---------------------\n')
print(classification_report(y_test, y_pred))
#################################################################################################################################

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Predict probabilities for the ROC curve
y_prob = model.predict_proba(X_test)[:, 1]  # Probability estimates for the positive class

# Compute the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, color='seagreen', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='midnightblue', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right',fontsize =16)
plt.show()

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)


# Plot confusion matrix
plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_ )[::-1]

fs = []
for thresh in thresholds:
    if thresh <=0:
        break
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)

    # train model
    selection_model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.01,
                           depth=6,l2_leaf_reg=3.0,
                           loss_function='MultiClass',rsm=0.8,verbose=100,
                          early_stopping_rounds=50 )
    selection_model.fit(select_X_train, y_train)

    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    accuracy = accuracy_score(y_test, y_pred)
    fs.append((thresh, select_X_train.shape[1], accuracy*100.0))
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))
    fs_df.to_csv('LICA_feature_selection.csv')

In [ ]:
# Merge df1 and df2 on 'ID'
df1 = pd.read_csv('RCCA_feature_selection.csv')
df2 = pd.read_csv('RICA_feature_selection.csv')
df3 = pd.read_csv('LCCA_feature_selection.csv')
df4 = pd.read_csv('LICA_feature_selection.csv')
merged_df = pd.merge(df1, df2, on='Feature Name', how='outer')

# Merge the result with df3
merged_df = pd.merge(merged_df, df3, on='Feature Name', how='outer')

# Merge the result with df4
merged_df = pd.merge(merged_df, df4, on='Feature Name', how='outer')

merged_df.to_csv('feature_selection.csv', index=False)